In [1]:
%matplotlib inline
import sys
sys.path.append("..")

In [2]:
import random

import deeptrack as dt

import numpy as np
import skimage.color
import matplotlib.pyplot as plt
import tensorflow as tf

tracker_input_size = 64


In [3]:
class CrescentMoon(dt.Ellipse):
    def get(self, *inp, radius, offset, **kwargs):
        ellipse = super().get(*inp, radius=radius, **kwargs)
        offset = np.round(offset).astype(np.int)
        other = dt.Affine(translate=offset, mode="constant").resolve(ellipse)
        ellipse[np.array(other > 0)] = 0
        
        return ellipse
        
class Rectangle(dt.Scatterer):
    def get(self, *inp, width, height, **kwargs):
        return np.ones((round(width), round(height), 1))
        

In [4]:

commons = dt.DummyFeature(
    image_size=tracker_input_size,
    position=lambda image_size: image_size / 2 + np.random.randn(2) * 2
)

arguments = dt.Arguments(
    noise_level=10,
    particle_type="moon"
)

fluorescence_microscope = dt.Fluorescence(
    NA=0.8,                
    resolution=1e-6,     
    magnification=10,
    wavelength=680e-9,
    output_region=(0, 0, tracker_input_size, tracker_input_size)
)

poisson_noise = dt.Poisson(
    snr=lambda: np.random.rand()*7 + 20,
    background=0.1
)


moon_particle = CrescentMoon(radius=1e-6, offset=(5, 0), position=commons.position)
rectangle_particle = Rectangle(height=18, width=12, position=commons.position)

particle = dt.ConditionalSetFeature(
    on_false=rectangle_particle,
    on_true=moon_particle,
    condition=lambda particle_type: particle_type == "moon",
    particle_type=arguments.particle_type
)

In [5]:


pipeline = (
    fluorescence_microscope(particle)
    >> dt.Affine(rotate=lambda:np.random.rand() * 2 * np.pi)
    >> dt.NormalizeMinMax() 
    + 0.2
) >> noise >> dt.NormalizeMinMax()

pipeline.bind_arguments(arguments)
pipeline.update().plot()

In [ ]:
dataset = [pipeline.update().resolve() for _ in range(1000)]
dataset_labels = [np.mean(d.get_property("rotate", get_one=False), axis=0) for d in dataset]

In [ ]:
symmetries = 2
key = "rect"
noise_level = 10

model = dt.models.AutoTracker(
    input_shape=(tracker_input_size, tracker_input_size, 1),
    mode="orientation",
    symmetries=symmetries,
)


model_data = dt.Value(pipeline.update().resolve(key=key, noise_level=noise_level))
model.fit(model_data, epochs=30, batch_size=8)

classification = [d.get_property("key") for d in dataset]
classes = set(classification)

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(model_data())
plt.axis("off")
plt.title("Model training sample")

plt.subplot(1, 2, 2)

for cl in classes:

    indexes = np.array(classification) == cl

    predicted_angle = model.predict(np.array(dataset))[indexes]
    predicted_angle_2 = model.predict(np.array(dataset)[:, ::-1])[indexes]

    angle = np.arctan2(predicted_angle[:, 1], predicted_angle[:, 0]) / symmetries
    angle_2 = np.arctan2(predicted_angle_2[:, 1], predicted_angle_2[:, 0]) / symmetries


    _dataset_labels = np.array(dataset_labels)[indexes] % ( 2 * np.pi / symmetries ) 

    diff = (angle - _dataset_labels)

    diff = (diff - np.mean(diff) + np.pi / symmetries) % (np.pi)
    diff = diff - np.mean(diff)

    xraw = np.array([d.get_property("snr") for d in dataset])[indexes]


    vmin = np.min(xraw)
    vmax = np.max(xraw)
    x = np.linspace(vmin, vmax, 10)

    _x = xraw
    _y = np.abs(diff)

    y = [np.mean( _y[(_x >= x[i]) & (_x < x[i+1]) ]) for i in range(9)]
    plt.plot(x[1:9], y[1:9])
plt.title("Orientation error")
plt.ylabel(f"Absolute prediction error")
plt.xlabel(f"Signal to noise")

plt.legend([x for x in classes])
plt.show()



NameError: name 'dt' is not defined